基于现有文本的序列，预处理成字符的序列：每 3 个字符一个序列，最高 60 个字符。

In [4]:
import keras
import numpy as np

path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text=open(path).read().lower()

print('Corpus length:', len(text))

Corpus length: 600893


In [5]:
maxlen = 60

step = 3

sentences = []
next_chars = []

for i in range(0, len(text) - maxlen - step):
    sentences.append(text[i:i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
# 进行 one-hot 编码
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    



Number of sequences: 600830
Unique characters: 57
Vectorization...


构建一个网络：LSTM + Dense + softMax。

In [7]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape = (maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

### 采样：对后续的字符进行预测：
 1. 给定当前的文本序列，从模型中得到下一个字符的概率。
 2. 根据给定的**温度**对概率重新进行加权（温度越高随机性越强，越有趣，温度越低越通顺）
 3. 根据新概率随机采样
 4. 将获取的字符追加到当前文本后

In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds . np.sum(exp_preds)
    # 根据概率值生成随机结果
    probas = np.random.multinomial(1, preds, 1) 
    return np.argmax(probas)

In [9]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    # 从随机位置找个种子
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index:start_index + maxlen]
    print('--- generating with seed: "', generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('--- temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)

epoch 1
4694/4694 [==============================] - 317s 68ms/step - loss: 1.7248
--- generating with seed: " (and indeed many kinds of sensations) are
to be recognized a"
--- temperature: 0.2
(and indeed many kinds of sensations) are
to be recognized a

AttributeError: 'numpy.ndarray' object has no attribute 'np'